# Add DocStrings and unit test to code

In [ ]:
!pip install pytest

In [ ]:
import os
import io
import sys
import pytest
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from google import genai
from google.genai import types

In [ ]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')
google_api_key = os.getenv('GOOGLE_API_KEY')
print(google_api_key)

In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
GEMINI_MODEL = "gemini-2.5-flash"
gemini = genai.Client(api_key=google_api_key)

In [ ]:
system_prompt_docstring_and_comment = """
You are a Python code documentation assistant. Your task is to read simple Python scripts consisting of a few functions and provide appropriately formatted comments and docstrings. Each function should have a short docstring describing its purpose, and inline comments should be added to break down the steps within the code. The comments should be concise and technical, suitable for a technical audience that will use the code to build unit tests. Return only the modified code with comments and docstrings, without any additional explanations or text.
"""

system_prompt_unit_test = """
You are a unit test generation assistant for Python code. Given a properly documented Python script with comments and docstrings, your task is to create relevant unit tests that thoroughly validate the functionality of each function. Aim for comprehensive test coverage, ensuring that edge cases and typical use cases are included. Generate the unit test script in a way that is executable as is, using the pytest framework. Return the generated unit test code without any additional explanations or text. The code should be executable as is, using the pytest framework. We should have clear output indicating the time spent, the percentage of test success and other details. The output should also include the original code so that the script can be run without external reference to the original source code.
"""

In [ ]:
def user_prompt_for_comments(python):
    user_prompt = """
    Analyze the following Python script. Document it by adding a short docstring for each function and inline comments to clarify the steps within the code. Return only the updated code with comments and docstrings
    {python}
    """
    user_prompt += python
    return user_prompt

In [ ]:
def user_prompt_for_unit_tests(python):
    user_prompt = """
    Analyze the following properly documented Python code and generate a full set of executable unit tests using the pytest framework. Ensure that the tests thoroughly check the functionality, including edge cases and typical scenarios. Return only the unit test code.The code should be executable as is, using the pytest framework. We should have clear output indicating the time spent, the percentage of test success and other details. The output should also include the original code so that the script can be run without external reference to the original source code. When executing code, I should see prints detailing the execution of the tests, how many tests are there how many passed how many failed and the execution time.
    {python}
    """
    user_prompt += python
    return user_prompt

In [ ]:
def generate_documentation(python):    
    messages = [{"role": "system", "content": system_prompt_docstring_and_comment},
                {"role": "user", "content": user_prompt_for_comments(python)}
               ]
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages= messages, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [ ]:
def generate_tests(python): 
    messages = [{"role": "system", "content": system_prompt_unit_test},
                {"role": "user", "content": user_prompt_for_unit_tests(python)}
               ]
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages= messages, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [ ]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("## Python Unit Test Generator")
    with gr.Row():
        with gr.Column():
            python_code = gr.TextArea(label="Python Code", placeholder="Enter your Python code here...")
            generate_doc_button = gr.Button("Generate Documentation")
        with gr.Column():
            doc_output = gr.TextArea(label="Generated Documentation code")
    with gr.Row():
        with gr.Column():
            generate_test_button = gr.Button("Generate Unit Tests")
            test_output = gr.TextArea(label="Generated Unit Tests")
            
        with gr.Column():
            execute_button = gr.Button("Execute Tests")
            execute_output = gr.TextArea(label="Test Execution Output")

    generate_test_button.click(fn=generate_tests, inputs=doc_output, outputs=test_output)
    execute_button.click(fn=execute_python, inputs=test_output, outputs=execute_output)
    generate_doc_button.click(fn=generate_documentation, inputs=python_code, outputs=doc_output)

ui.launch(inbrowser=True)